In [13]:
import pandas as pd
import re
import os
from tqdm import tqdm
from transformers import pipeline
import huggingface_hub

In [2]:
dfs = []

folder_path = "c:/Users/User/Documents/NIR3/files/Сообщения из соцсетей/"

for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        
        with open(file_path, 'r', encoding='utf-8') as f:
            first_line = f.readline()
        
        sep = ',' if first_line.count(',') > first_line.count(';') else ';'
        
        temp_df = pd.read_csv(file_path, sep=sep, on_bad_lines='skip')
        temp_df["file_name"] = file  
        dfs.append(temp_df)

if dfs:
    df = pd.concat(dfs, ignore_index=True)

df["Ссылка"] = "https://vk.com/" + df["file_name"].str[:-4]

C:\Users\User\AppData\Local\Temp\ipykernel_6556\1522476861.py:14: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(file_path, sep=sep, on_bad_lines='skip')
C:\Users\User\AppData\Local\Temp\ipykernel_6556\1522476861.py:14: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(file_path, sep=sep, on_bad_lines='skip')
C:\Users\User\AppData\Local\Temp\ipykernel_6556\1522476861.py:14: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(file_path, sep=sep, on_bad_lines='skip')
C:\Users\User\AppData\Local\Temp\ipykernel_6556\1522476861.py:14: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(file_path, sep=sep, on_bad_lines='skip')
C:\Users\User\AppData\Local\Temp\ipykernel_6556\1522476861.py:14: DtypeWarni

In [3]:
df

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack,file_name,Unnamed: 0,Ссылка
0,2024-05-23 22:04:17,778816,Санкт-Петербург занял второе место в рейтинге ...,1166.0,58,1.0,post,NaN,NaN,NaN,a_beglov.csv,NaN,https://vk.com/a_beglov
1,2024-05-23 10:14:27,778633,«Время героев» – уникальная кадровая программа...,13653.0,250,35.0,post,NaN,NaN,NaN,a_beglov.csv,NaN,https://vk.com/a_beglov
2,2024-05-22 14:43:14,778359,Хорошие новости для жителей Фрунзенского район...,33078.0,225,28.0,post,NaN,NaN,NaN,a_beglov.csv,NaN,https://vk.com/a_beglov
3,2024-05-22 09:39:44,778173,По поручению Президента России создаём на Пулк...,26973.0,623,156.0,post,NaN,NaN,NaN,a_beglov.csv,NaN,https://vk.com/a_beglov
4,2024-05-21 16:09:56,777954,Расскажу о наиболее заметных событиях федераль...,24784.0,252,9.0,post,NaN,NaN,NaN,a_beglov.csv,NaN,https://vk.com/a_beglov
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3026643,2022-01-07 03:56:03,24229,"[id349998486|Владимир], поняла. Спасибо))",NaN,0,NaN,reply,NaN,23852.0,23865.0,zhit_v_moskovskom.csv,NaN,https://vk.com/zhit_v_moskovskom
3026644,2022-01-07 12:40:03,24234,"[id276339910|Анастасия Уш],здравствуйте. С 1 я...",NaN,0,NaN,reply,NaN,23852.0,23865.0,zhit_v_moskovskom.csv,NaN,https://vk.com/zhit_v_moskovskom
3026645,2021-12-28 19:08:53,23907,Добрый день! Будет ли организован Раздельный с...,NaN,0,NaN,comment,NaN,23852.0,NaN,zhit_v_moskovskom.csv,NaN,https://vk.com/zhit_v_moskovskom
3026646,2021-12-28 10:55:10,23850,Дай Бог случится,NaN,0,NaN,comment,NaN,23845.0,NaN,zhit_v_moskovskom.csv,NaN,https://vk.com/zhit_v_moskovskom


In [4]:
# df = pd.read_csv(r'c:\Users\User\Documents\NIR3\files\Сообщения из соцсетей\club59160781.csv', sep=';')
df_links = pd.read_excel(r'c:\Users\User\Documents\NIR3\files\Сообщества ВК для сбора данных.xlsx')

In [5]:

def clean_text(text):

    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  
    text = re.sub(r'[A-Za-z]', '', text)
    text = re.sub(r'\d+', '', text) 
    text = re.sub(r'\n', ' ', text)
    return text

df = df[df['text'].str.len() >= 10]
df['text'] = df['text'].apply(clean_text)

C:\Users\User\AppData\Local\Temp\ipykernel_6556\2940856519.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(clean_text)


In [6]:
df = df.merge(df_links[['Ссылка', 'Район']], on='Ссылка', how='left')

In [11]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
df = df[(df['date'] <= '2022-12-30 00:00:00')]
df = df[(df['date'] >= '2021-12-31 23:59:59')]
df = df.sort_values(by='date')

In [16]:
df

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack,file_name,Unnamed: 0,Ссылка,Район
1725825,2022-01-01 00:00:04,10631,дорогие соседи друзья и подписчикинаше сообщес...,1590.0,43,3.0,post,NaN,NaN,NaN,sestroreck.home.csv,NaN,https://vk.com/sestroreck.home,Курортный район
2831915,2022-01-01 00:00:08,37569,с новым годом любимый курортный,1766.0,40,3.0,post,NaN,NaN,NaN,vesti_kurort.csv,NaN,https://vk.com/vesti_kurort,Курортный район
851156,2022-01-01 00:00:10,229877,с наступившим новым годом друзья пусть эта н...,850.0,37,3.0,post,NaN,NaN,NaN,kronshtadt2000.csv,NaN,https://vk.com/kronshtadt2000,Кронштадтский район
1056921,2022-01-01 00:00:12,3435,с новым годом дорогие пороховчане,1074.0,48,1.0,post,NaN,NaN,NaN,mo.porohovie.csv,NaN,https://vk.com/mo.porohovie,Красногвардейский
1045024,2022-01-01 00:00:28,1269003,екатерина возможно очень много информации ране...,NaN,0,NaN,reply,NaN,1268969.0,1268981.0,kupchinonews.csv,NaN,https://vk.com/kupchinonews,Фрунзенский район
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636176,2022-12-29 23:45:05,1084524,больница имени филатова ая,NaN,2,NaN,comment,NaN,1084522.0,NaN,kolpino.csv,NaN,https://vk.com/kolpino,Колпинский
636177,2022-12-29 23:46:19,1084525,уверяю вас врач от бога вытащила нас с затяжн...,NaN,1,NaN,comment,NaN,1084522.0,NaN,kolpino.csv,NaN,https://vk.com/kolpino,Колпинский
1613460,2022-12-29 23:47:27,1529955,скорее транспортных средств,NaN,0,NaN,comment,NaN,1529466.0,NaN,pushkin_story.csv,NaN,https://vk.com/pushkin_story,Пушкинский район
1236576,2022-12-29 23:47:29,107959,электричка тоже дорожает руб будет до км пр...,NaN,1,NaN,reply,NaN,107804.0,107951.0,passtransportspb.csv,NaN,https://vk.com/passtransportspb,Комитет по транспорту


In [19]:
repository_id = 'Glebosol/sm_problem_cat'
classifier = pipeline("sentiment-analysis", model=repository_id, tokenizer=repository_id, device=0)
MAX_LENGTH = 2048 

Device set to use cuda:0


In [20]:
def classify_text(text):
    text = text[:MAX_LENGTH]
    result = classifier(text)[0]
    return result['label'], result['score']


tqdm.pandas(desc="Классификация текста")
df[['label', 'score']] = df['text'].progress_apply(lambda x: pd.Series(classify_text(x)))

Классификация текста: 100%|██████████| 1354158/1354158 [1:17:36<00:00, 290.82it/s]


In [24]:
df

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack,file_name,Unnamed: 0,Ссылка,Район,label,score
1045227,2022-01-01 00:02:35,1269004,пишут петарды плохо ну тогда и новый год зачем...,NaN,1,NaN,reply,NaN,1268748.0,1268757.0,kupchinonews.csv,NaN,https://vk.com/kupchinonews,Фрунзенский район,LABEL_1,0.997522
1045025,2022-01-01 00:03:22,1269005,владимир более чем реально выкручиваются из шл...,NaN,3,NaN,reply,NaN,1268969.0,1268981.0,kupchinonews.csv,NaN,https://vk.com/kupchinonews,Фрунзенский район,LABEL_1,0.972523
2823992,2022-01-01 00:08:14,1065802,не уверена видно что кот домашний попробовали ...,NaN,2,NaN,reply,NaN,1065800.0,1065801.0,veselyposelok.csv,NaN,https://vk.com/veselyposelok,Невский,LABEL_1,0.996989
1044974,2022-01-01 00:11:38,1269006,ирина согласна у меня кот и то час в ванной пр...,NaN,4,NaN,reply,NaN,1268969.0,1268970.0,kupchinonews.csv,NaN,https://vk.com/kupchinonews,Фрунзенский район,LABEL_1,0.995949
1045003,2022-01-01 00:16:58,1269007,вера мой тоже в ванне но если на улицу то стро...,NaN,1,NaN,reply,NaN,1268969.0,1268971.0,kupchinonews.csv,NaN,https://vk.com/kupchinonews,Фрунзенский район,LABEL_1,0.996381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338967,2022-12-29 23:39:14,1366597,градоначальник едет на уборку снега,NaN,0,NaN,comment,NaN,1366522.0,NaN,grazhdanka_spb.csv,NaN,https://vk.com/grazhdanka_spb,Калининский,LABEL_1,0.994646
1236575,2022-12-29 23:43:45,107958,сергей по пушкину и в колпино ездить нормальн...,NaN,1,NaN,reply,NaN,107804.0,107951.0,passtransportspb.csv,NaN,https://vk.com/passtransportspb,Комитет по транспорту,LABEL_1,0.986338
636177,2022-12-29 23:46:19,1084525,уверяю вас врач от бога вытащила нас с затяжн...,NaN,1,NaN,comment,NaN,1084522.0,NaN,kolpino.csv,NaN,https://vk.com/kolpino,Колпинский,LABEL_1,0.959681
1236576,2022-12-29 23:47:29,107959,электричка тоже дорожает руб будет до км пр...,NaN,1,NaN,reply,NaN,107804.0,107951.0,passtransportspb.csv,NaN,https://vk.com/passtransportspb,Комитет по транспорту,LABEL_1,0.995953


In [22]:
df['label'].value_counts()

label
LABEL_1    1068295
LABEL_0     285863
Name: count, dtype: int64

In [23]:
df = df[(df['label'] == 'LABEL_1')]
df = df[(df['score'] >= 0.95)]

In [26]:
repository_id = 'Glebosol/spb_city_prob_class'
classifier = pipeline("sentiment-analysis", model=repository_id, tokenizer=repository_id, device=0)
MAX_LENGTH = 2048 

Device set to use cuda:0


In [27]:
tqdm.pandas(desc="Классификация текста")
df[['label_cat', 'score_cat']] = df['text'].progress_apply(lambda x: pd.Series(classify_text(x)))

Классификация текста: 100%|██████████| 863480/863480 [48:47<00:00, 294.97it/s]  


In [28]:
df

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack,file_name,Unnamed: 0,Ссылка,Район,label,score,label_cat,score_cat
1045227,2022-01-01 00:02:35,1269004,пишут петарды плохо ну тогда и новый год зачем...,NaN,1,NaN,reply,NaN,1268748.0,1268757.0,kupchinonews.csv,NaN,https://vk.com/kupchinonews,Фрунзенский район,LABEL_1,0.997522,"Благоустройство,",0.635471
1045025,2022-01-01 00:03:22,1269005,владимир более чем реально выкручиваются из шл...,NaN,3,NaN,reply,NaN,1268969.0,1268981.0,kupchinonews.csv,NaN,https://vk.com/kupchinonews,Фрунзенский район,LABEL_1,0.972523,"Благоустройство,",0.504010
2823992,2022-01-01 00:08:14,1065802,не уверена видно что кот домашний попробовали ...,NaN,2,NaN,reply,NaN,1065800.0,1065801.0,veselyposelok.csv,NaN,https://vk.com/veselyposelok,Невский,LABEL_1,0.996989,"Отходы,",0.376402
1044974,2022-01-01 00:11:38,1269006,ирина согласна у меня кот и то час в ванной пр...,NaN,4,NaN,reply,NaN,1268969.0,1268970.0,kupchinonews.csv,NaN,https://vk.com/kupchinonews,Фрунзенский район,LABEL_1,0.995949,Энергетика,0.570718
1045003,2022-01-01 00:16:58,1269007,вера мой тоже в ванне но если на улицу то стро...,NaN,1,NaN,reply,NaN,1268969.0,1268971.0,kupchinonews.csv,NaN,https://vk.com/kupchinonews,Фрунзенский район,LABEL_1,0.996381,"ЖКХ,",0.555702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338967,2022-12-29 23:39:14,1366597,градоначальник едет на уборку снега,NaN,0,NaN,comment,NaN,1366522.0,NaN,grazhdanka_spb.csv,NaN,https://vk.com/grazhdanka_spb,Калининский,LABEL_1,0.994646,"Дороги,",0.897382
1236575,2022-12-29 23:43:45,107958,сергей по пушкину и в колпино ездить нормальн...,NaN,1,NaN,reply,NaN,107804.0,107951.0,passtransportspb.csv,NaN,https://vk.com/passtransportspb,Комитет по транспорту,LABEL_1,0.986338,"Транспорт,",0.917523
636177,2022-12-29 23:46:19,1084525,уверяю вас врач от бога вытащила нас с затяжн...,NaN,1,NaN,comment,NaN,1084522.0,NaN,kolpino.csv,NaN,https://vk.com/kolpino,Колпинский,LABEL_1,0.959681,"Здравоохранение,",0.984308
1236576,2022-12-29 23:47:29,107959,электричка тоже дорожает руб будет до км пр...,NaN,1,NaN,reply,NaN,107804.0,107951.0,passtransportspb.csv,NaN,https://vk.com/passtransportspb,Комитет по транспорту,LABEL_1,0.995953,"Транспорт,",0.974681


In [29]:
df.to_csv(r'c:\Users\User\Documents\NIR3\files\Обработанные_сообщения.csv', sep=';', index=False)